<a href="https://colab.research.google.com/github/ZackTanCZ/Expenses_Analysis/blob/main/Google_Sheet_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Libraries
import json, re, datetime as dt
import pandas as pd, numpy as np

# Python library to access google sheets - https://docs.gspread.org/en/latest/
import gspread
import google.auth
from google.oauth2 import service_account

In [ ]:
# Grab the Service Account Credentials from the JSON file
creds = service_account.Credentials.from_service_account_file('vocal-catalyst-359907-2523fbe4a84c.json')

# Include the scope
scoped_creds = creds.with_scopes(['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive'])

# authorize the clientsheet with gspread
client = gspread.authorize(scoped_creds)

In [ ]:
# Regular expression for matching spreadsheet - e.g. '[Jan'25] Living Expenses'
# title_regex = re.compile(r"\[[A-Za-z][A-Za-z][A-Za-z]+'[0-9][0-9]+\] Living Expenses", re.IGNORECASE)
title_regex = re.compile(r"\[[A-Za-z][A-Za-z][A-Za-z]+'24+\] Living Expenses", re.IGNORECASE)

# print(title_regex.search('[Jun\'24] Living Expenses'))

In [ ]:
# # Handling of single spreadsheet
# for spreadsheet in client.list_spreadsheet_files():
#   if(title_regex.search(spreadsheet['name'])):
#     print(spreadsheet['name'])

#     # Opening the spreadsheet with matching name
#     matching_sheet = client.open(spreadsheet['name'])

#     print(matching_sheet.worksheets())

#     # Select the 2nd worksheet - Transactions
#     data_worksheet = matching_sheet.worksheet('Transactions')

#     # Extract the data from cells B4:E100
#     record_list = data_worksheet.get("B4:E100")

#     # Slice the list into column header and row records
#     col_header, col_data = record_list[0], record_list[1:]

#     # Cast into a DataFrame for Data Engineering
#     gSheetDF = pd.DataFrame(col_data, columns = col_header)

#     # Remove '$' sign in the 'Amount' Column
#     gSheetDF['Amount'] = gSheetDF['Amount'].str.slice(1)

#     # Cast the DataFrame into the appropriate datatype
#     gSheetDF = gSheetDF.astype({"Date": str, "Amount": float,
#                           "Description": str, "Category": str})

#     # # Dont cast to datetime yet. Do it after creating the dataset
#     # # Cast 'Date' column into datetime
#     # gSheetDF['Date'] = pd.to_datetime(gSheetDF['Date'], format='%d/%m/%Y')


In [ ]:
# display(gSheetDF)

In this iteration, we assume that there's only one google spreadsheet to extract data from. What if there are multiple spreadsheets to extract and collate our data from?

the "for" loop will to be amended to account for multi-DataFrames.

One approach is to


*   Store each individual DataFrame into an empty list
*   Use pd.concat() to append each DataFrame into one grand DataFrame




In [ ]:
# Handling of Multiple spreadsheet
spreadsheet_df_list = [] # An empty list

for spreadsheet in client.list_spreadsheet_files():
  if(title_regex.search(spreadsheet['name'])):
    print(spreadsheet['name'])

    # Opening the spreadsheet with matching name
    matching_sheet = client.open(spreadsheet['name'])

    print(matching_sheet.worksheets())

    # Select the 2nd worksheet 'Transactions' with the .worksheet method
    data_worksheet = matching_sheet.worksheet('Transactions')

    # Extract the data from cells B4:E100
    record_list = data_worksheet.get("B4:E100")

    # Slice the list into column header and row records
    col_header, col_data = record_list[0], record_list[1:]

    # Cast into a DataFrame for Data Engineering
    gSheetDF = pd.DataFrame(col_data, columns = col_header)

    # Remove '$' sign in the 'Amount' Column
    gSheetDF['Amount'] = gSheetDF['Amount'].str.slice(1)

    # Cast the DataFrame into the appropriate datatype
    gSheetDF = gSheetDF.astype({"Date": str, "Amount": float,
                          "Description": str, "Category": str})

    # # Dont cast to datetime yet. Do it after creating the dataset
    # # Cast 'Date' column into datetime
    # gSheetDF['Date'] = pd.to_datetime(gSheetDF['Date'], format='%d/%m/%Y')

    # Append each DataFrame into the 'spreadsheet_df_list' list
    spreadsheet_df_list.append(gSheetDF)


[May'24] Living Expenses
[<Worksheet 'Summary' id:0>, <Worksheet 'Transactions' id:1732160294>]
[Apr'24] Living Expenses
[<Worksheet 'Summary' id:0>, <Worksheet 'Transactions' id:1732160294>]
[Mar'24] Living Expenses
[<Worksheet 'Summary' id:0>, <Worksheet 'Transactions' id:1732160294>]
[Feb'24] Living Expenses
[<Worksheet 'Summary' id:0>, <Worksheet 'Transactions' id:1732160294>]
[Jan'24] Living Expenses
[<Worksheet 'Summary' id:0>, <Worksheet 'Transactions' id:1732160294>]
[Jun'24] Living Expenses
[<Worksheet 'Summary' id:0>, <Worksheet 'Transactions' id:1732160294>]
[Jul'24] Living Expenses
[<Worksheet 'Summary' id:0>, <Worksheet 'Transactions' id:1732160294>]


In [ ]:
print(len(spreadsheet_df_list))

7


In [ ]:
# Concat each DataFrame found in the list into one final DataFrame
final_df = pd.concat(spreadsheet_df_list, ignore_index=True)
display(final_df)

,Date,Amount,Description,Category
0,01/05/2024,50.0,Rent,Home
1,01/05/2024,11.0,Dinner,Food
2,02/05/2024,4.2,Dinner,Food
3,03/05/2024,20.0,Dinner,Food
4,04/05/2024,6.0,Dinner,Food
...,...,...,...,...
254,28/7/2024,13.6,Dinner,Food
255,29/7/2024,18.6,Dinner,Food
256,30/7/2024,10.2,Dinner,Food
257,31/7/2024,2.5,ActiveSG Gym Pass,Personal


In [ ]:
# # get the instance of the Spreadsheet
# sheet = client.open('[Jan\'25] Living Expenses')

In [ ]:
# # # get the title of all worksheet - returns a list
# # worksheet_list = sheet.worksheets()
# # print(worksheet_list)

# # Extract the 'Transaction' worksheet
# trans_sheet = sheet.get_worksheet(1)

# # get all the records from 'trans_sheet'
# record_list = trans_sheet.get("B4:E100")


In [ ]:
# # Convert the list of list into a panda dataframe
# gSheetDF = pd.DataFrame(record_list[1:], columns = record_list[0])


In [ ]:
# # Data Cleaning

# # Remove '$' sign in the 'Amount' Column
# gSheetDF['Amount'] = gSheetDF['Amount'].str.slice(1)

# # Cast the DataFrame into the appropriate datatype
# gSheetDF = gSheetDF.astype({"Date": str, "Amount": float,
#                       "Description": str, "Category": str})

# # # Dont cast to datetime yet. Do it after creating the dataset
# # # Cast 'Date' column into datetime
# # gSheetDF['Date'] = pd.to_datetime(gSheetDF['Date'], format='%d/%m/%Y')


# Saving the DataFrame into a new spreadsheet

After collating our data from google spreadsheets, ***one single spreadsheet*** is used store the data. The spreadsheet - 'expensesDataSheet' - is created if it doesn't exist. Otherwise, it is reused to avoid duplicates.

The idea is to use ***PowerBI*** and query for this googlesheet for Visualisation

In [ ]:
# https://stackoverflow.com/questions/75935222/copy-google-sheet-with-service-account-and-then-transfer-ownership-to-a-differen

# 1. Grab the spreadsheet shared with the Service Account
sheetname = 'Test_Sheet_0602' # Change this for testing
sheetDict = client.list_spreadsheet_files(sheetname)

if (len(sheetDict)) == 0:
  # 1a. If the spreadsheet doesn't exist
  # Creates the spreadsheet using the service account
  sh = client.create(sheetname)

  # Share the spreadsheet with my actual account - Set the role as "writer" first
  res = sh.share('tan.cz1993@gmail.com', perm_type='user', role="writer", notify=True)

  # Retrieve the permission id to identify my account
  print(res.json()) # Display the content of the json file
  permissionId = res.json()["id"]

  # Transfer the ownership of the spreadsheet to my actual account
  sh.transfer_ownership(permissionId)

  # Now, I have to manually accept ownership of the spreadsheet - log in via gmail
  print(f'Please log in to your gmail account and accept ownership of the spreadsheet - {sheetname}')
else:
  # 1b. if the spreadsheet does exist
  # Open the spreadsheet
  sh = client.open(sheetname)

  # List all the permission associated with the spreadsheet
  print('Spreadsheet found! Listing all the associated permissions')
  for listing in sh.list_permissions():
    email, role = listing['emailAddress'], listing['role']
    print(f'Email: {email}')
    print(f'Role: {role}')


Spreadsheet found! Listing all the associated permissions
Email: tan.cz1993@gmail.com
Role: owner
Email: tcz-701@vocal-catalyst-359907.iam.gserviceaccount.com
Role: writer


In [ ]:
# 2. Select the worksheet from the spreadsheet - expensesDataSheet
worksheet = sh.sheet1  # Or sh.worksheet("Sheet Name") if you want a specific sheet

# 3. Clear existing data in the selected sheet (optional but often useful)
worksheet.clear()

{'spreadsheetId': '1Q81a-_PYfavuh3Vv2cnqp_tQMlR9Xwdu80cD0SX-EjU',
 'clearedRange': 'Sheet1!A1:Z1000'}

In [ ]:
# 4. Convert DataFrame to a list of lists (required by gspread)
data = [final_df.columns.values.tolist()] + final_df.values.tolist()  # Include header row
# print(data)


In [ ]:
# 5. Update the sheet with the DataFrame data, starting the cell 'A1'
worksheet.update(data, 'A1')

{'spreadsheetId': '1Q81a-_PYfavuh3Vv2cnqp_tQMlR9Xwdu80cD0SX-EjU',
 'updatedRange': 'Sheet1!A1:D260',
 'updatedRows': 260,
 'updatedColumns': 4,
 'updatedCells': 1040}